# Extract from JSON and XML

You'll now get practice extracting data from JSON and XML. You'll extract the same population data from the previous exercise, except the data will be in a different format.

Both JSON and XML are common formats for storing data. XML was established before JSON, and JSON has become more popular over time. They both tend to be used for sending data via web APIs, which you'll learn about later in the lesson.

Sometimes, you can obtain the same data in either JSON or XML format. That is the case for this exercise. You'll use the same data except one file is formatted as JSON and the other as XML.

There is a solution file for these exercises. Go to File->Open and click on 2_extract_exercise_solution.ipynb.

# Extract JSON and JSON Exercise

First, you'll practice extracting data from a JSON file. Run the cell below to print out the first line of the JSON file.

In [ ]:
### 
#   Run the following cell.
#   This cell loads a function that prints the first n lines of
#   a file.
#
#   Then this function is called on the JSON file to print out
#   the first line of the population_data.json file
###

def print_lines(n, file_name):
    f = open(file_name)
    for i in range(n):
        print(f.readline())
    f.close()
    
print_lines(1, 'population_data.json')

The first "line" in the file is actually the entire file. JSON is a compact way of representing data in a dictionary-like format. Luckily, pandas has a method to [read in a json file](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_json.html). 

If you open the link with the documentation, you'll see there is an *orient* option that can handle JSON formatted in different ways:
```
'split' : dict like {index -> [index], columns -> [columns], data -> [values]}
'records' : list like [{column -> value}, ... , {column -> value}]
'index' : dict like {index -> {column -> value}}
'columns' : dict like {column -> {index -> value}}
'values' : just the values array
```

In this case, the JSON is formatted with a 'records' orientation, so you'll need to use that value in the read_json() method. You can tell that the format is 'records' by comparing the pattern in the documentation with the pattern in the JSON file.

Next, read in the population_data.json file using pandas.

In [ ]:
# TODO: Read in the population_data.json file using pandas's 
# read_json method. Don't forget to specific the orient option
# store the rsults in df_json

import pandas as pd
df_json = pd.read_json('population_data.json', orient='records')

In [ ]:
# TODO: Use the head method to see the first few rows of the resulting
# dataframe
df_json.head()

Notice that this population data is the same as the data from the previous exercise. The column order might have changed, but the data is otherwise the same.

# Other Ways to Read in JSON

Besides using pandas to read JSON files, you can use the json library. Run the code cell below to see an example of reading in JSON with the json library. Python treats JSON data like a dictionary.

In [ ]:
import json

# read in the JSON file
with open('population_data.json') as f:
    json_data = json.load(f)

# print the first record in the JSON file
print(json_data[0])
print('\n')

# show that JSON data is essentially a dictionary
print(json_data[0]['Country Name'])
print(json_data[0]['Country Code'])

# Extract XML

Next, you'll work with the same data except now the data is in xml format. Run the next code cell to see what the first fifteen lines of the data file look like.

In [ ]:
print_lines(15, 'population_data.xml')

XML looks very similar to HTML. XML is formatted with tags with values inside the tags. XML is not as easy to navigate as JSON. Pandas cannot read in XML directly. One reason is that tag names are user defined. Every XML file might have different formatting. You can imagine why XML has fallen out of favor relative to JSON.

### How to read and navigate XML

There is a Python library called BeautifulSoup, which makes reading in and parsing XML data easier. Here is the link to the documentation: [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/)

The find() method will find the first place where an xml element occurs. For example using find('record') will return the first record in the xml file:

```xml
<record>
  <field name="Country or Area" key="ABW">Aruba</field>
  <field name="Item" key="SP.POP.TOTL">Population, total</field>
  <field name="Year">1960</field>
  <field name="Value">54211</field>
</record>
```

The find_all() method returns all of the matching tags. So find_all('record') would return all of the elements with the `<record>` tag.

Run the code cells below to get a basic idea of how to navigate XML with BeautifulSoup. To navigate through the xml file, you search for a specific tag using the find() method or find_all() method. 

Below these code cells, there is an exercise for wrangling the XML data.

In [2]:
# import the BeautifulSoup library
from bs4 import BeautifulSoup

# open the population_data.xml file and load into Beautiful Soup
with open("population_data.xml") as fp:
    soup = BeautifulSoup(fp, "lxml") # lxml is the Parser type

UnicodeDecodeError: 'cp950' codec can't decode byte 0xbf in position 2: illegal multibyte sequence

In [3]:
# output the first 5 records in the xml file
# this is an example of how to navigate with BeautifulSoup

i = 0
# use the find_all method to get all record tags in the document
for record in soup.find_all('record'):
    # use the find_all method to get all fields in each record
    i += 1
    for record in record.find_all('field'):
        print(record['name'], ': ' , record.text)
    print()
    if i == 5:
        break

NameError: name 'soup' is not defined

# XML Exercise (Challenge)

Create a data frame from the xml file. This exercise is somewhat tricky. One solution would be to convert the xml data into dictionaries and then use the dictionaries to create a data frame. 

The dataframe should have the following layout:

| Country or Area | Year | Item | Value |
|----|----|----|----|
| Aruba | 1960 | Population, total | 54211 |
| Aruba | 1961 | Population, total | 55348 |
etc...

Technically, extracting XML, transforming the results, and putting it into a data frame is a full ETL pipeline. This exercise is jumping ahead in terms of what's to come later in the lesson. But it's a good chance to familiarize yourself with XML. 

In [1]:
# TODO: Create a pandas data frame from the XML data.
# HINT: You can use dictionaries to create pandas data frames.
# HINT: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.from_dict.html#pandas.DataFrame.from_dict
# HINT: You can make a dictionary for each column or for each row
# HINT: Modify the code from the previous code cell


# output the first 5 records in the xml file
# this is an example of how to navigate with BeautifulSoup

# use the find_all method to get all record tags in the document
data_dictionary = {'Country or Area':[], 'Year':[], 'Item':[], 'Value':[]}

for record in soup.find_all('record'):
    for record in record.find_all('field'):
        data_dictionary[record['name']].append(record.text)

df = pd.DataFrame.from_dict(data_dictionary)
df = df.pivot(index='Country or Area', columns='Year', values='Value')
df.reset_index(level=0, inplace=True)

NameError: name 'soup' is not defined

# Conclusion

Like CSV, JSON and XML are ways to format data. If everything is formatted correctly, JSON is especially easy to work with. XML is an older standard and a bit trickier to handle.

As a reminder, there is a solution file for these exercises. You can go to File->Open and then click on 2_extract_exercise.